In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.

In [ ]:
# Import necessary libraries
import os
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Directories setup
base_dir = 'Dataset/'  # Adjust path if necessary
MildDemented_Dir = os.path.join(base_dir, 'Mild_Demented')
ModerateDemented_Dir = os.path.join(base_dir, 'Moderate_Demented')
NonDemented_Dir = os.path.join(base_dir, 'Non_Demented')
VeryMildDemented_Dir = os.path.join(base_dir, 'Very_Mild_Demented')

# Prepare dataset paths
filepaths = []
labels = []
dict_list = [MildDemented_Dir, ModerateDemented_Dir, NonDemented_Dir, VeryMildDemented_Dir]
class_labels = ['Mild Demented', 'Moderate Demented', 'Non Demented', 'Very Mild Demented']

for i, dir_path in enumerate(dict_list):
    file_list = os.listdir(dir_path)
    for f in file_list:
        filepaths.append(os.path.join(dir_path, f))
        labels.append(class_labels[i])

Alzheimer_df = pd.DataFrame({'filepaths': filepaths, 'labels': labels})

# Train-test split
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(Alzheimer_df, test_size=0.2, stratify=Alzheimer_df['labels'], random_state=42)

# Image augmentation
IMG_HEIGHT, IMG_WIDTH = 224, 224  # Increased resolution
BATCH_SIZE = 32
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True
)
valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filepaths',
    y_col='labels',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical',
    batch_size=BATCH_SIZE
)
valid_generator = valid_datagen.flow_from_dataframe(
    valid_df,
    x_col='filepaths',
    y_col='labels',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

# Model creation with Transfer Learning and Fine-tuning
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
for layer in base_model.layers[:-4]:  # Freeze all but the last 4 layers
    layer.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes
])

# Compile model with Learning Rate Scheduler
lr_schedule = Adam(learning_rate=1e-3)
model.compile(optimizer=lr_schedule, loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping and saving best model
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_alzheimer_model.h5', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(train_generator,
                    validation_data=valid_generator,
                    epochs=20,
                    callbacks=[early_stop, checkpoint])

# Save the model
model.save('final_alzheimer_model.h5')

FileNotFoundError: [Errno 2] No such file or directory: 'Dataset/Mild_Demented'

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define a simple CNN model (adjust as needed)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),  # Adjust input shape
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  # Assuming you have 4 classes
])

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))  # Match the input shape of your model
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image
    return img_array

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))  # Match the input shape of your model
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image
    return img_array

In [ ]:
def predict(image_path):
    image = load_and_preprocess_image(image_path)  # Load and preprocess the image
    print(f"Input shape: {image.shape}")  # Debugging line
    predictions = model.predict(image)  # Make predictions
    class_index = np.argmax(predictions)  # Get the index of the highest prediction
    return class_labels[class_index]  # Return the predicted class

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
# After training your model
model.save('alzheimer_detection_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('alzheimer_detection_model.h5')

In [ ]:
import os
import numpy as np
import gradio as gr
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing import image

# Load your pre-trained model
model = load_model('final_alzheimer_model.h5')

# Compile the model (if needed, based on your original training configuration)
optimizer = Adam(learning_rate=1e-3)  # Adjust the learning rate as necessary
loss_function = SparseCategoricalCrossentropy()  # Adjust based on your original loss function
metrics = ['accuracy']  # Add any additional metrics you were using
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Define your class labels (adjust according to your dataset)
class_labels = ['Non Demented', 'Very Mild Demented', 'Mild Demented', 'Moderate Demented']

# Function to load and preprocess the image
def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))  # Adjust the target size based on your model
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image
    return img_array

# Your prediction function
def predict(image_path):
    # Load and preprocess the image
    image = load_and_preprocess_image(image_path)
    # Make predictions
    predictions = model.predict(image)
    # Print raw prediction probabilities for debugging
    print("Raw Predictions:", predictions)
    # Interpret predictions
    class_index = np.argmax(predictions)
    return class_labels[class_index]

# Setup the Gradio interface
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="filepath"),  # Use gr.Image for newer versions
    outputs="text",
    title="Alzheimer's Disease Detection",
    description="Upload an MRI image to detect Alzheimer's disease."
)

# Launch the interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f31b895d1fb708fcde.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import numpy as np
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load your pre-trained binary classification model
model = load_model('final_alzheimer_model.h5')

# Define your image preprocessing function
def load_and_preprocess_image(image_path):
    try:
        # Load the image, resize, and normalize it
        image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))  # Change target_size as needed
        image = tf.keras.preprocessing.image.img_to_array(image)
        image = np.expand_dims(image, axis=0)  # Add batch dimension
        image /= 255.0  # Normalize to [0, 1]
        return image
    except Exception as e:
        return f"Error loading image: {e}"

# Your prediction function
def predict(image_path):
    try:
        # Load and preprocess the image
        image = load_and_preprocess_image(image_path)
        if isinstance(image, str):  # Check if the error message was returned
            return image

        # Make predictions
        predictions = model.predict(image)

        # Print the raw predictions for debugging
        print("Raw predictions:", predictions)

        # Interpret predictions
        class_index = np.argmax(predictions, axis=-1)  # Get index of the highest probability
        print("Class index:", class_index)

        if predictions[0][0] >= 0.2:  # If you have a sigmoid output
            return "Alzheimer's Disease Detected"
        else:
            return "Not Detected"
    except Exception as e:
        return f"Prediction error: {e}"

# Setup the Gradio interface
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="filepath"),
    outputs="text",
    title="Alzheimer's Disease Detection",
    description="Upload an MRI image to detect Alzheimer's disease."
)

# Launch the interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://57caeb0f403dd7ed95.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
